### Name: Javad Hezareh
### Std.Num: 98101074

# **Preface**
In this notebook you are going to implement a powerful blackbox attack namely [NES](https://arxiv.org/pdf/1804.08598.pdf) and test it on a regular target model, then you will use a simple but powerful defense called  Random Noise Defense ([RND](https://arxiv.org/pdf/2104.11470.pdf)) against this attack on the same target model in order to check the power of the aforementioned attack in the presence of this so-called defense.
You will have to use a **CIFAR10 ResNet18 model** as the target model for this set of experiments.
You may want to train a new model or load an already trained model, your choice, but this model must have at least 94% accuracy on CIFAR10 test set as told in the previous exercises.

P.S. Don't normalize the data used for training.


In [1]:
!pip install gdown

In [2]:
!gdown --id 1p3cBO3dqOQ9dtb0W7IfG-HGhK4wEB3u1

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1p3cBO3dqOQ9dtb0W7IfG-HGhK4wEB3u1
From (redirected): https://drive.google.com/uc?id=1p3cBO3dqOQ9dtb0W7IfG-HGhK4wEB3u1&confirm=t&uuid=78324ce8-536f-4766-93d0-655ab7e550ea
To: /kaggle/working/model.ptk
100%|██████████████████████████████████████| 44.8M/44.8M [00:00<00:00, 49.1MB/s]


In [3]:
resnet_url = "https://raw.githubusercontent.com/kuangliu/pytorch-cifar/master/models/resnet.py"
!wget $resnet_url

--2023-06-15 10:19:39--  https://raw.githubusercontent.com/kuangliu/pytorch-cifar/master/models/resnet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4218 (4.1K) [text/plain]
Saving to: ‘resnet.py’

resnet.py           100%[===================>]   4.12K  --.-KB/s    in 0s      

2023-06-15 10:19:39 (32.1 MB/s) - ‘resnet.py’ saved [4218/4218]



In [4]:
import torch
import numpy as np
import torch as ch
import torchvision
import torchvision.transforms.functional as F
import torchvision.transforms as transforms
import argparse
import json
import pdb
import os
import time
import math
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import DataParallel
from torch.nn.modules import Upsample
from matplotlib import pyplot as plt
from tqdm import tqdm

# from google.colab import drive
# drive.mount('/content/drive/')

# resnet_url = "https://raw.githubusercontent.com/kuangliu/pytorch-cifar/master/models/resnet.py"
# !wget $resnet_url

from resnet import *

In [5]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

100%|██████████| 170498071/170498071 [00:02<00:00, 85193949.84it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


'cuda'

In [6]:
resnet = ResNet18().to(device)
resnet.load_state_dict(torch.load('./model.ptk'))
correct = 0
total = 0

resnet.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Standard Accuracy of ResNet18 model on the {len(testset)} test images: {100 * correct / total} %')

Standard Accuracy of ResNet18 model on the 10000 test images: 94.1 %


# **Natural Evolutionary Strategies (NES)**

Firstly complete the code for the attack itself and then the add_Gaussian method used for the Random Noise Defense in the NES attack class.

In [15]:
CLASSIFIERS = {
    "resnet18": (ResNet18, 32),
    # "resnet18": (models.resnet502, 224),
}

NUM_CLASSES = {
    "cifar10": 10,
}

# Set the path to your dataset
CIFAR10_PATH = "./data"

if CIFAR10_PATH == "":
  raise ValueError("Please fill out the path to Cifar10!")

class NES():

  def __init__(self, model_to_fool, args, dataset_size, RND_coefficient, device):
      if args != None:
        self.args = args
      else:
        self.max_queries = 10000
        self.fd_eta = 0.01
        self.image_lr = 0.01
        self.mode = "linf"
        self.json_config = None
        self.epsilon = 0.05
        self.batch_size = 10
        self.log_process = "store_true"
        self.gradient_iters = 15
        self.total_images = 10000
        self.classifier = 'resnet18'
        self.args = {
            'max_queries': self.max_queries,
            'fd_eta': self.fd_eta,
            'image_lr': self.image_lr,
            'mode': self.mode,
            'json_config': self.json_config,
            'epsilon': self.epsilon,
            'batch_size': self.batch_size,
            'log_progress': self.log_progress,
            'gradient_iters': self.gradient_iters,
            'total_images': self.total_images,
            'classifier': self.classifier,
        }
      self.model_to_fool = model_to_fool
      self.dataset_size = dataset_size
      self.RND_coefficient = RND_coefficient
      self.device = device
      if self.device == 'cuda':
        ch.set_default_tensor_type('torch.cuda.FloatTensor')
      else:
        ch.set_default_tensor_type('torch.FloatTensor')

      if self.dataset_size == 32:
        self.channel_size = 3
      else:
        raise ValueError("Please check out the dataset_size parameter...")


  """For setting up the Random Noise Defense"""

  def add_Gausian(self, input_image, coeff):
    # Firstly define a random epsilon with the input image shape
    # Remember that it should be a float
    # Add the epsilon with the proper coefficient to the input image
    # Don't forget to clamp it between the right values
    # P.S. Remember to convert everything to the right device
    #############################
    # Your code goes here
    e = ch.randn_like(input_image).to(self.device).float()
    noisy_image = input_image + coeff * e
    return ch.clamp(noisy_image, 0, 1)
    #############################

  def generate(self):
      dataset = None
      if self.dataset_size == 32:
        transform = transforms.Compose([transforms.ToTensor()])#, transforms.Resize(self.dataset_size), transforms.CenterCrop(self.dataset_size),])
        dataset = torchvision.datasets.CIFAR10(root = CIFAR10_PATH, train = False, download = False, transform = transform)
      else:
        raise ValueError("Please check out the dataset_size parameter...")

      dataset_loader = DataLoader(dataset, batch_size=self.args["batch_size"])

      total_correct, total_adv, total_queries = 0, 0, 0
      with tqdm(enumerate(dataset_loader), total=len(dataset_loader)) as pbar:
          for i, (images, targets) in pbar:
            if i*self.args["batch_size"] >= self.args["total_images"]:
                break

            res = self.make_adversarial_examples(images.to(self.device), targets.to(self.device), self.args, self.model_to_fool, self.dataset_size)
            ncc = res['num_correctly_classified'] # Number of correctly classified images (originally)
            num_adv = ncc * res['success_rate'] # Success rate was calculated as (# adv)/(# correct classified)
            queries = num_adv * res['average_queries'] # Average queries was calculated as (total queries for advs)/(# advs)
            total_correct += ncc
            total_adv += num_adv
            total_queries += queries
            print(f'ncc: {ncc}\t|\tnum_adv: {num_adv}')
            pbar.set_description(f"batch: {i}|")

      print("-"*80)
      if total_adv != 0:
        print("Final Success Rate: {succ} | Final Average Queries for Successful Adv Examples: {aq}".format(
                aq=int(total_queries/total_adv),
                succ=total_adv/total_correct))
      else:
        """If total adversarial examples is zero then we won't print any avg number of queries for success
        because we had no success and the avg number of queries has exceeded the budget"""
        print("Final Success Rate: {succ} ".format(
                # aq = total_queries,
                succ=total_adv/total_correct))
      print("-"*80)
    #   return self.adversarial_images

  def norm(self, t):
      assert len(t.shape) == 4
      norm_vec = ch.sqrt(t.pow(2).sum(dim=[1,2,3])).view(-1, 1, 1, 1)
      norm_vec += (norm_vec == 0).float()*1e-8
      return norm_vec

  ###
  # Different optimization steps
  # All take the form of func(x, g, lr)
  # eg: exponentiated gradients
  # gd: gradient descent
  # l2/linf: projected gradient descent
  ###


  def eg_step(self, x, g, lr):
      real_x = (x + 1)/2 # from [-1, 1] to [0, 1]
      pos = real_x*ch.exp(lr*g)
      neg = (1-real_x)*ch.exp(-lr*g)
      new_x = pos/(pos+neg)
      return new_x*2-1

  def linf_step(self, x, g, lr):
      return x + lr*ch.sign(g)

  def l2_prior_step(self, x, g, lr):
      new_x = x + lr*g/self.norm(g)
      norm_new_x = self.norm(new_x)
      norm_mask = (norm_new_x < 1.0).float()
      return new_x*norm_mask + (1-norm_mask)*new_x/norm_new_x

  def gd_prior_step(self, x, g, lr):
      return x + lr*g

  def l2_image_step(self, x, g, lr):
      return x + lr*g/self.norm(g)

  ##
  # Projection steps for l2 and linf constraints:
  # All take the form of func(new_x, old_x, epsilon)
  ##

  def l2_proj(self, image, eps):
      orig = image.clone()
      def proj(new_x):
          delta = new_x - orig
          out_of_bounds_mask = (self.norm(delta) > eps).float()
          x = (orig + eps*delta/self.norm(delta))*out_of_bounds_mask
          x += new_x*(1-out_of_bounds_mask)
          return x
      return proj

  def linf_proj(self, image, eps):
      orig = image.clone()
      def proj(new_x):
          return orig + ch.clamp(new_x - orig, -eps, eps)
      return proj

  ##
  # Main functions
  ##

  def make_adversarial_examples(self, image, true_label, args, model_to_fool, DATASET_SIZE):
      '''
      The main process for generating adversarial examples with priors.
      '''
      # Initial setup
      prior_size = DATASET_SIZE
      upsampler = Upsample(size=(DATASET_SIZE, DATASET_SIZE))
      total_queries = ch.zeros(args["batch_size"])
      prior = ch.zeros(args["batch_size"], self.channel_size, prior_size, prior_size)
      dim = prior.nelement()/args["batch_size"]
      image_step = self.l2_image_step if args["mode"] == 'l2' else self.linf_step
      proj_maker = self.l2_proj if args["mode"] == 'l2' else self.linf_proj
      proj_step = proj_maker(image, args["epsilon"])

      # Loss function
      criterion = ch.nn.CrossEntropyLoss(reduction='none')

      def normalized_eval(x):
          x_copy = x.clone()

          # You can comment or uncomment the code needed for a specific case
          """################## Normal prediction ##################"""
#           x_copy = ch.clamp(x_copy, 0, 1)
#           return self.model_to_fool(x_copy)

          """################## Prediction using random noise defense ##################"""
          x_copy_noisy = self.add_Gausian(x_copy, self.RND_coefficient)
          x_copy_noisy = ch.clamp(x_copy_noisy, 0, 1)
          return self.model_to_fool(x_copy_noisy)

      L = lambda x: criterion(normalized_eval(x), true_label)

      losses = L(image)

      # Original classifications
      orig_images = image.clone()

      # You can comment or uncomment the code needed for a specific case
      """##################Normal prediction##################"""
#       orig_classes = self.model_to_fool(orig_images).argmax(1).to(self.device)

      """################## Prediction using random noise defense ##################"""
      image_noisy = self.add_Gausian(image, self.RND_coefficient)
      image_noisy = ch.clamp(image_noisy, 0, 1)
      orig_classes = self.model_to_fool(image_noisy).argmax(1).to(self.device)

      correct_classified_mask = (orig_classes == true_label).float()
      total_ims = correct_classified_mask.sum()
      not_dones_mask = correct_classified_mask.clone()

      t = 0
      while not ch.any(total_queries > args["max_queries"]):
          t += args["gradient_iters"]*2
          if t >= args["max_queries"]:
              break

          prior = ch.zeros_like(image)
          for _ in range(args["gradient_iters"]):

              # Define a tensor of random Gaussian noise with the input image shape
              # Then devide this tensor by (dim**0.5)
              # Now add and subtract the defined noise with the fd_eta coefficient with the original image
              # Finally build the gradient estimation of the loss using the finite difference method
              #############################
              # Your code goes here
              exp_noise = ch.randn(size=image.shape).to(self.device) / (dim**0.5)
              q1 = image + args['fd_eta'] * exp_noise
              q2 = image - args['fd_eta'] * exp_noise

              est_deriv = (L(q1) - L(q2))/args['fd_eta']
              #############################

              prior += est_deriv.view(-1, 1, 1, 1)*exp_noise

          # Preserve images that are already done,
          # Unless we are specifically measuring gradient estimation
          prior = prior*not_dones_mask.view(-1, 1, 1, 1)

          ## Update the image:
          # take a pgd step using the prior
          new_im = image_step(image, upsampler(prior*correct_classified_mask.view(-1, 1, 1, 1)), args["image_lr"])
          image = proj_step(new_im)
          image = ch.clamp(image, 0, 1)

        #   if args["mode"] == 'l2':
        #       if not ch.all(self.norm(image - orig_images) <= args["epsilon"] + 1e-3):
        #           pass
        #           # pdb.set_trace()
        #   else:
        #       if not (image - orig_images).max() <= args["epsilon"] + 1e-3:
        #         pass
        #           # pdb.set_trace()

          ## Continue query count
          total_queries += 2*args["gradient_iters"]*not_dones_mask
          not_dones_mask = not_dones_mask*((normalized_eval(image).argmax(1) == true_label).float())

          ## Logging and stuff

          # new_losses = L(image)
          success_mask = (1 - not_dones_mask)*correct_classified_mask
          num_success = success_mask.sum()
          if num_success != 0:
            success_queries = ((success_mask*total_queries).sum()/num_success).cpu().item()
          else:
            success_queries = ((success_mask*total_queries).sum()).cpu().item()
          """If num_success == 0 it means that the number of successful adv examples was zero (No adv examples could be made!)
          so success queries must not be printed """

          current_success_rate = (num_success/correct_classified_mask.sum()).cpu().item()
          #not_done_loss = ((new_losses*not_dones_mask).sum()/not_dones_mask.sum()).cpu().item()
          max_curr_queries = total_queries.max().cpu().item()
          if args["log_progress"]:
            print("Queries: %d (%d) | Success rate: %f | Average queries: %f" % (max_curr_queries , t, current_success_rate, success_queries))

          if current_success_rate == 1.0:
              break

      return {
              'average_queries': success_queries,
              'num_correctly_classified': correct_classified_mask.sum().cpu().item(),
              'success_rate': current_success_rate,
              'images_orig': orig_images.cpu().numpy(),
              'images_adv': image.cpu().numpy(),
              'all_queries': total_queries.cpu().numpy(),
              'correctly_classified': correct_classified_mask.cpu().numpy(),
              'success': success_mask.cpu().numpy()
      }

# **Attack Arguments Setup**

**Write a report and indicate the success rate and average number of qureies for sections A and B.**

A) Now you must launch the attack using different values;
For the whole CIFAR10 test set, first on the regular target model and without the Random Noise Defense, with fd_eta parameter of 0.01 and 0.001, using "linf" mode (2 results).

B) Then apply the Random Noise Defense on the target model with RND_coefficient of 0.01 and 0.02 against NES with fd_eta parameter of 0.01 and 0.001 again using the "linf" mode (4 results).

Analyse and compare the results of both sections in your report.


In [16]:
budget = 1000
target_model_arch = 'resnet18'
dataset_size = 32

args = {
    'max_queries': budget,
    'fd_eta': 0.01,
    'image_lr': 0.01,
    'mode': 'linf',
    'json_config': None,
    'epsilon': 0.05,
    'batch_size': 500,
    'log_progress': "store_true",
    'gradient_iters': 15,
    'total_images': 10000,
    'classifier': target_model_arch,
}

target_model = resnet
target_model.eval()
RND_coefficients = [0, 0.01, 0.02]
fd_eta_values = [0.01, 0.001]


with torch.no_grad():
    # Launch the attack using the NES class and its generate method
    #############################
    # Your code goes here
    for rnd_coeff in RND_coefficients:
        for fd_eta in fd_eta_values:
            args['fd_eta'] = fd_eta
            print(f"Parameters:\tfd_eta={args['fd_eta']}\tRND_coeff={rnd_coeff}")
            nes_generator = NES(target_model, args, dataset_size, rnd_coeff, device)
            nes_generator.generate()
            print()
    #############################

Parameters:	fd_eta=0.01	RND_coeff=0


  0%|          | 0/20 [00:00<?, ?it/s]

Queries: 30 (30) | Success rate: 0.057816 | Average queries: 30.000000
Queries: 60 (60) | Success rate: 0.126338 | Average queries: 46.271187
Queries: 90 (90) | Success rate: 0.237687 | Average queries: 66.756760
Queries: 120 (120) | Success rate: 0.319058 | Average queries: 80.335571
Queries: 150 (150) | Success rate: 0.404711 | Average queries: 95.079369
Queries: 180 (180) | Success rate: 0.468951 | Average queries: 106.712326
Queries: 210 (210) | Success rate: 0.535332 | Average queries: 119.519997
Queries: 240 (240) | Success rate: 0.580300 | Average queries: 128.856094
Queries: 270 (270) | Success rate: 0.648822 | Average queries: 143.762375
Queries: 300 (300) | Success rate: 0.678801 | Average queries: 150.662460
Queries: 330 (330) | Success rate: 0.706638 | Average queries: 157.727280
Queries: 360 (360) | Success rate: 0.719486 | Average queries: 161.339279
Queries: 390 (390) | Success rate: 0.732334 | Average queries: 165.350876
Queries: 420 (420) | Success rate: 0.751606 | Ave

batch: 0|:   5%|▌         | 1/20 [02:04<39:28, 124.66s/it]

Queries: 990 (990) | Success rate: 0.899358 | Average queries: 247.571426
ncc: 467.0	|	num_adv: 420.0000070929527
Queries: 30 (30) | Success rate: 0.053305 | Average queries: 30.000000
Queries: 60 (60) | Success rate: 0.136461 | Average queries: 48.281250
Queries: 90 (90) | Success rate: 0.228145 | Average queries: 65.046730
Queries: 120 (120) | Success rate: 0.298507 | Average queries: 78.000000
Queries: 150 (150) | Success rate: 0.360341 | Average queries: 90.355026
Queries: 180 (180) | Success rate: 0.417910 | Average queries: 102.704079
Queries: 210 (210) | Success rate: 0.471215 | Average queries: 114.841629
Queries: 240 (240) | Success rate: 0.530917 | Average queries: 128.915665
Queries: 270 (270) | Success rate: 0.577825 | Average queries: 140.369003
Queries: 300 (300) | Success rate: 0.609808 | Average queries: 148.741257
Queries: 330 (330) | Success rate: 0.648188 | Average queries: 159.473679
Queries: 360 (360) | Success rate: 0.673774 | Average queries: 167.088608
Queries: 

...

Queries: 990 (990) | Success rate: 0.274298 | Average queries: 333.779541
ncc: 463.0	|	num_adv: 126.9999934732914
Queries: 30 (30) | Success rate: 0.032609 | Average queries: 30.000000
Queries: 60 (60) | Success rate: 0.065217 | Average queries: 45.000000
Queries: 90 (90) | Success rate: 0.082609 | Average queries: 54.473682
Queries: 120 (120) | Success rate: 0.097826 | Average queries: 64.666664
Queries: 150 (150) | Success rate: 0.106522 | Average queries: 71.632652
Queries: 180 (180) | Success rate: 0.126087 | Average queries: 88.448273
Queries: 210 (210) | Success rate: 0.141304 | Average queries: 101.538460
Queries: 240 (240) | Success rate: 0.147826 | Average queries: 107.647057
Queries: 270 (270) | Success rate: 0.160870 | Average queries: 120.810814
Queries: 300 (300) | Success rate: 0.165217 | Average queries: 125.526314
Queries: 330 (330) | Success rate: 0.173913 | Average queries: 135.750000
Queries: 360 (360) | Success rate: 0.184783 | Average queries: 148.941177
Queries: 3

batch: 18|:  95%|█████████▌| 19/20 [39:29<02:04, 124.69s/it]

Queries: 990 (990) | Success rate: 0.289130 | Average queries: 323.909760
ncc: 460.0	|	num_adv: 133.00000667572021
Queries: 30 (30) | Success rate: 0.017316 | Average queries: 30.000000
Queries: 60 (60) | Success rate: 0.036797 | Average queries: 45.882355
Queries: 90 (90) | Success rate: 0.054113 | Average queries: 60.000000
Queries: 120 (120) | Success rate: 0.086580 | Average queries: 82.500000
Queries: 150 (150) | Success rate: 0.103896 | Average queries: 93.750000
Queries: 180 (180) | Success rate: 0.116883 | Average queries: 103.333336
Queries: 210 (210) | Success rate: 0.123377 | Average queries: 108.947365
Queries: 240 (240) | Success rate: 0.136364 | Average queries: 121.428574
Queries: 270 (270) | Success rate: 0.142857 | Average queries: 128.181824
Queries: 300 (300) | Success rate: 0.153680 | Average queries: 140.281693
Queries: 330 (330) | Success rate: 0.160173 | Average queries: 147.972977
Queries: 360 (360) | Success rate: 0.170996 | Average queries: 161.392410
Queries:

batch: 19|: 100%|██████████| 20/20 [41:34<00:00, 124.72s/it]

Queries: 990 (990) | Success rate: 0.259740 | Average queries: 309.250000
ncc: 462.0	|	num_adv: 120.00000160932541
--------------------------------------------------------------------------------
Final Success Rate: 0.2778749478598922 | Final Average Queries for Successful Adv Examples: 320
--------------------------------------------------------------------------------



## Results

Parameters:	fd_eta=0.01	RND_coeff=0

Final Success Rate: 0.9034006389117266 | Final Average Queries for Successful Adv Examples: 261

---

Parameters:	fd_eta=0.001	RND_coeff=0

Final Success Rate: 0.867481406607866 | Final Average Queries for Successful Adv Examples: 258

---

Parameters:	fd_eta=0.01	RND_coeff=0.01

Final Success Rate: 0.2321008757635757 | Final Average Queries for Successful Adv Examples: 337

---

Parameters:	fd_eta=0.001	RND_coeff=0.01

Final Success Rate: 0.2171727963480254 | Final Average Queries for Successful Adv Examples: 329

---

Parameters:	fd_eta=0.01	RND_coeff=0.02

Final Success Rate: 0.2813388771886665 | Final Average Queries for Successful Adv Examples: 310

---

Parameters:	fd_eta=0.001	RND_coeff=0.02

Final Success Rate: 0.2778749478598922 | Final Average Queries for Successful Adv Examples: 320